# Detect the ones who are only screened 

> run eligibility notebook

> run survey data notebook



In [2]:
%run "../eligibility.ipynb"
%run "../survey_data/survey_data.ipynb"

In [74]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
survey_data.shape
eligibility.shape

(50032, 19)

(37398, 36)

In [75]:
survey_data.columns
eligibility.columns

Index(['case_id', 'organisation', 'first_name', 'last_name', 'dob',
       'interview_date', 'adress', 'commune', 'commune_section',
       'interviewer_firstname', 'interviewer_lastname', 'is_your_parent_alive',
       'mothers_name', 'fathers_name', 'who_is_your_law_parent', 'total',
       'is_eligible_for_dep_hiv', 'age', 'already_in_a_group'],
      dtype='object')

Index(['case_id', 'dreams_code', 'organisation', 'id', 'phone',
       'not_selectable', 'first_name', 'last_name', 'dob', 'interview_date',
       'adress', 'commune', 'commune_section', 'interviewer_firstname',
       'interviewer_lastname', 'is_your_parent_alive', 'mothers_name',
       'fathers_name', 'total', 'who_is_your_law_parent', 'group_name',
       'id_group', 'hub_name', 'id_hub', 'eske_ou_lekol_deja',
       'depi_kile_ou_pa_al_lekol_with_label',
       'c61_depi_ki_l_ou_pa_al_lekl_ank', 'c6_ske_ou_te_oblije_double_deja',
       'kiyes_ki_peye_lekol_pou_ou', 'age', 'kpi_score', 'age_range',
       'newage_range', 'date_entevyou', 'fiscal_year', 'timeOn_system'],
      dtype='object')

## Eligibility

In [77]:
eligibility.dreams_code.fillna("null",inplace=True)

In [79]:
def tranche_age_classique(age):
        if age>=10 and age<=14:
            return "10-14"
        elif age>=15 and age<=19:
            return "15-19"
        elif age>=20 and age<=24:
            return "20-24"
        elif age>=25 and age<=29:
            return "25-29"
        else:
            return "not_valid_age"
        
def tranche_age_mineur_majeur(age):
        if age>=10 and age<=17:
            return "10-17"
        elif age>=18 and age<=19:
            return "18-19"
        elif age>=20 and age<=24:
            return "20-24"
        elif age>=25 and age<=29:
            return "25-29"
        else:
            return "not_valid_age"

In [80]:
eligibility['age_range'] = eligibility.age.map(tranche_age_classique)
eligibility['newage_range'] = eligibility.age.map(tranche_age_mineur_majeur)

In [82]:
import pandas as pd
from datetime import datetime

In [83]:
eligibility["date_entevyou"] = pd.to_datetime( eligibility.interview_date)

In [84]:
def fiscalYear21(date):
    if date.year == 2021 and date.month>=1 and date.month<=3:
        return "FY21Q2"
    elif date.year == 2020 and date.month>=10 and date.month<=12:
        return "FY21Q1"
    elif date.year == 2021 and date.month>=4 and date.month<=6:
        return "FY21Q3"
    elif date.year == 2021 and date.month>=7 and date.month<=9:
        return "FY21Q4"
    else:
        return "not_valid_fy"
    

    

def validTimeOnSystem(date):
    if date>= datetime.strptime("2020-04-01","%Y-%m-%d") and date<= datetime.now():
        return "required_Time_on"
    else:
        return "not_valid_time_on"


In [85]:
eligibility["fiscal_year"] = eligibility.date_entevyou.map(fiscalYear21)
eligibility["timeOn_system"] = eligibility.date_entevyou.map(validTimeOnSystem)

In [86]:
almost_eligibility = eligibility[(eligibility.dreams_code == "null")&
                                 (eligibility.timeOn_system=="required_Time_on")&
                                 (eligibility.age_range!="25-29")&
                                 (eligibility.age_range!="not_valid_age")]

In [87]:
almost_eligibility.dreams_code.count()

4391

In [88]:
def firstlastdob(df):
    return f"{df.first_name} {df.last_name} {df.dob}"

In [89]:
%%capture
almost_eligibility["unduplicate_me"] = almost_eligibility.apply(lambda df: firstlastdob(df),axis=1)
clean_eligibility =  almost_eligibility.drop_duplicates(subset=["unduplicate_me"])


In [90]:
clean_eligibility.dreams_code.count()

4360

## Survey data

In [95]:
survey_data["remove_duplicate"] = survey_data.apply(lambda df: firstlastdob(df),axis=1)

In [98]:
survey_data["date_entevyou"] = pd.to_datetime( survey_data.interview_date)
survey_data["fiscal_year"] = survey_data.date_entevyou.map(fiscalYear21)
survey_data["timeOn_system"] = survey_data.date_entevyou.map(validTimeOnSystem)
survey_data['age_range'] = survey_data.age.map(tranche_age_classique)
survey_data['newage_range'] = survey_data.age.map(tranche_age_mineur_majeur)

In [102]:
almost_survey_data = survey_data[(survey_data.already_in_a_group == "no")&
                                 (survey_data.timeOn_system=="required_Time_on")&
                                 (survey_data.age_range!="25-29")&
                                 (survey_data.age_range!="not_valid_age")&
                                 (survey_data.total>=14)]

In [106]:
almost_survey_data.case_id.count()

4390

In [104]:
clean_survey_data =  almost_survey_data.drop_duplicates(subset=["remove_duplicate"])

In [105]:
clean_survey_data.case_id.count()

4359

## what we want

In [110]:
id_form_survey_data = clean_survey_data[["case_id","already_in_a_group"]]

In [117]:
update_screening_data = id_form_survey_data.merge(clean_eligibility, on="case_id",how='right')

In [145]:
update_screening_data.case_id.count()

4360

In [121]:
update_screening_data.to_excel("update_screened.xlsx",index=False,na_rep="NULL")

## verification des 5757 Mastersheet

In [125]:
#%run "../layeringOfServices.ipynb"

In [127]:
old_5757 = pd.read_excel("./old/liste_filles_screenees_non_desservies_1avril2020_now.xlsx")
mastersheet = pd.read_excel("../LayeringOF_services.xlsx")

In [128]:
old_5757.case_id.count()

5757

In [129]:
mastersheet.case_id.count()

23864

In [132]:
# update - old
old_version = old_5757[["case_id","dreams_code","already_in_a_group"]]


In [134]:
update_version = update_screening_data[["case_id","dreams_code","already_in_a_group"]]

In [175]:
situationOU = old_version.merge(update_version,on="case_id",how="left")
situationOU.dreams_code_y.fillna(0,inplace=True)
esk = situationOU[situationOU.dreams_code_y==0]


In [176]:
dreams =mastersheet[["case_id","code","recevoir_1services"]]

In [179]:
%%capture
dreams.fillna(-1,inplace=True)

In [181]:
casdreams = dreams.merge(esk,on="case_id",how="left")
casdreams.dreams_code_x.fillna(10,inplace=True)
casdreams[casdreams.dreams_code_x!=10]

,case_id,code,recevoir_1services,dreams_code_x,already_in_a_group_x,dreams_code_y,already_in_a_group_y
21400,39ca88c4-5dc4-4f42-b44f-62fa34033555,GON/DRMS/086237501,1_services_dreams_recus,A_NULL,NaN,0,NaN
21401,7923ff56-1e80-4e05-a7ef-a922c7456ba9,GON/DRMS/086237502,1_services_dreams_recus,A_NULL,NaN,0,NaN
21402,de8efc6a-bd9b-4b8a-beca-7e88d02dbd42,GON/DRMS/086237503,1_services_dreams_recus,A_NULL,NaN,0,NaN
21404,f48ead4e-634e-4741-8075-1db28021743b,GON/DRMS/086237505,1_services_dreams_recus,A_NULL,NaN,0,NaN
21405,f66a5a89-08dd-4d54-a536-a6213b9190e0,GON/DRMS/086237506,1_services_dreams_recus,A_NULL,NaN,0,NaN
...,...,...,...,...,...,...,...
23859,72d1375a-8bc0-4206-9450-151e78fef814,CAP/DRMS/021252405,1_services_dreams_recus,A_NULL,no,0,NaN
23860,8b22c3ab-0188-4899-a454-fd10d7dfb345,CAP/DRMS/021252406,1_services_dreams_recus,A_NULL,no,0,NaN
23861,8e6cbcf6-d205-42ac-997f-fdd00ec40afe,CAP/DRMS/021252407,1_services_dreams_recus,A_NULL,no,0,NaN
23862,adf2c4ec-ad01-44a2-89ba-3c02a604265a,CAP/DRMS/021252408,1_services_dreams_recus,A_NULL,no,0,NaN
